In [1]:
from fpml import resolve_template
from r4_r6_us_core_mapping_templates import templates
import json
from pathlib import Path
from git import Repo
import yaml

#===Globals===
repo_path = '/Users/ehaas/Documents/FHIR/US-Core'
target_branch = 'R6_prototype'
examples_yaml_path = Path(r'/Users/ehaas/Documents/FHIR/R6_Sandbox/uscore-v8-examples-yaml')
save_to_file = True

print(examples_yaml_path)

/Users/ehaas/Documents/FHIR/R6_Sandbox/uscore-v8-examples-yaml


### Write File to US Core R6 build for profile validation

- write to the R6 prototype build.
  - Check the Current US Core Branch
  - If not the Current Branch print notice

In [2]:
def write_to_repo(json_obj,file_name):
  repo = Repo(repo_path)
  # Get the current branch name
  current_branch = repo.active_branch.name
  print(f"Current branch: {current_branch}")
  if current_branch == target_branch:
    file_path = Path(repo_path) / 'input' / 'examples-yaml' / file_name
    print(f'writing to US Core {file_path}...')
    file_path.write_text(yaml.dump(json_obj, indent=2, sort_keys=False))
  else:
    print(f"Not on US Core's target branch '{target_branch}'")

### Scrub empty dicts and lists from mapped example

In [3]:
def remove_empty_lists_and_dicts(d):
    if not isinstance(d, dict):
        return d
    result = {}
    for k, v in d.items():
        if isinstance(v, dict):
            # Recursively process nested dictionaries
            v = remove_empty_lists_and_dicts(v)
            # Only include if the processed dictionary is not empty
            if v:
                result[k] = v
        elif isinstance(v, list) and all(isinstance(i, dict) for i in v):
            # Process lists of dictionaries
            v = [remove_empty_lists_and_dicts(i) for i in v]
            # Only include non-empty dictionaries from the list
            v = [i for i in v if i]
            # Only include the list if it's not empty
            if v:
                result[k] = v
        elif not (isinstance(v, list) and len(v) == 0):
            # Include non-empty lists and other non-empty values
            result[k] = v
    return result


### Fetch US Core examples
- Fetch US Core V8 examples
  -  downloaded from the master GitHub repro source YAML files to preserve the titles and descriptions
- If template for type is available then convert to R6 using the FHIRPathMapper Module
  - templates is dict with type as key to enable fetching examples by type
  -  TODO: Consider update to fetch templates by profile
     - evaluate how those profiles would be different and whether validation is enough or custom templating would be better

In [5]:

for uscore_example in examples_yaml_path.glob("*.yml"):
    json_obj = yaml.load(uscore_example.read_text(), Loader=yaml.SafeLoader)
    try:
      resource_type = json_obj['resourceType']
    except KeyError:
      resource_type = None
    # print(uscore_example.name, resource_type)
    try:
      r6_template = (templates[resource_type])
      # print(type(r6_template))
      R6_mapping = resolve_template(json_obj, r6_template)
    except KeyError:
        # print("============== NO TEMPLATE FOUND==============")
        pass
    else:
      scrubbed_R6_mapping = remove_empty_lists_and_dicts(R6_mapping)
      print(f"======= R6 {uscore_example.name}, Type = {resource_type} ========")
      # print(json.dumps(scrubbed_R6_mapping, indent=2))
      # print('============= YAML ================')
      print(yaml.dump(scrubbed_R6_mapping, indent=2, sort_keys=False))
      if save_to_file:
         write_to_repo(scrubbed_R6_mapping,uscore_example.name)
         

======= R6 allergyintolerance-example.yml, Type = AllergyIntolerance ========
resourceType: AllergyIntolerance
id: example
meta:
  extension:
  - url: http://hl7.org/fhir/StructureDefinition/instance-name
    valueString: AllergyIntolerance Example
  - url: http://hl7.org/fhir/StructureDefinition/instance-description
    valueMarkdown: This is a allergyintolerance example for the *US Core AllergyIntolerance
      Profile*.
  profile:
  - http://hl7.org/fhir/us/core/StructureDefinition/us-core-allergyintolerance
clinicalStatus:
  coding:
  - system: http://terminology.hl7.org/CodeSystem/allergyintolerance-clinical
    code: active
verificationStatus:
  coding:
  - system: http://terminology.hl7.org/CodeSystem/allergyintolerance-verification
    code: confirmed
category:
- medication
criticality: high
code:
  coding:
  - system: http://snomed.info/sct
    version: http://snomed.info/sct/731000124108
    code: '763875007'
    display: Product containing sulfonamide (product)
  text: sulfo